# Task2: User Engagement Analysis

In [83]:
# import requiered python libraries

import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go  
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn import preprocessing
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, normalize

In [88]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append('../scripts/')
from scripts.dataframe_outliers import *
import scripts.plots_file

importing Jupyter notebook from C:\Users\Machine\Documents\Telecom-Data-Analysis\scripts\plots_file.ipynb


In [89]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [90]:
# load and read the cleaned data
df_cleaned = pd.read_csv('../data/cleaned_Telecom_data.csv')
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149001 entries, 0 to 149000
Data columns (total 46 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      149001 non-null  int64  
 1   Bearer Id                       149001 non-null  float64
 2   Start                           149001 non-null  object 
 3   Start ms                        149001 non-null  float64
 4   End                             149001 non-null  object 
 5   End ms                          149001 non-null  float64
 6   Dur. (ms)                       149001 non-null  float64
 7   IMSI                            149001 non-null  float64
 8   MSISDN/Number                   149001 non-null  float64
 9   IMEI                            149001 non-null  float64
 10  Last Location Name              149001 non-null  object 
 11  Avg RTT DL (ms)                 149001 non-null  float64
 12  Avg RTT UL (ms) 

In [93]:
#Aggregate the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric
df_user_engage =  df_cleaned[['MSISDN/Number', 'Bearer Id', 'Dur. (ms)', 'Total DL (Bytes)']].copy().rename(columns={
    'Bearer_Id': 'Session_Frequency', 'MSISDN/Number':'Customer_Id', 'Dur. (ms)': 'Duration', 'Total DL (Bytes)': 'Total_Data_Volume'})